In [ ]:
import os
import pycisTopic
from dotenv import load_dotenv
from pathlib import Path
import pandas as pd
import numpy as np

load_dotenv()

out_dir = Path(os.getenv("OUTPUT_PATH")) / "garcia_ATAC/atac_preprocessing_combined"

In [ ]:
import pickle
cistopic_obj = pickle.load(open(os.path.join(out_dir, "cistopic_obj.pkl"), "rb"))
fragment_matrix = cistopic_obj.fragment_matrix

In [ ]:
fragment_matrix

In [ ]:
# Create a dataframe that has cell_id as rows, peaks as columns (or vice versa)
df_counts = pd.DataFrame.sparse.from_spmatrix(fragment_matrix, index=cistopic_obj.region_names, columns=cistopic_obj.cell_names)

# Merge cell_type and sample info
cell_info = cistopic_obj.cell_data[["sample", "celltype"]]

# For each cell_type–sample combo, sum the counts
pseudobulk_counts = []
for (ctype, smp), subset_ix in cell_info.groupby(['celltype', 'sample']).groups.items():
    summed_counts = df_counts.loc[:, subset_ix].sum(axis=1)
    # Name that row something like ctype_sample
    pseudobulk_counts.append(summed_counts.rename(f'{ctype}X{smp}'))

# Combine into a single pseudobulk df
df_pseudobulk = pd.concat(pseudobulk_counts, axis=1)


In [ ]:
df_pseudobulk

In [ ]:
df_pseudobulk.to_csv(out_dir / "pseudobulk_export.csv", sep = ",")

In [ ]:
test = pd.read_csv(out_dir / "pseudobulk_export.csv", index_col=0)

In [ ]:
test